# README

This notebook was created to solve a list of Spark challenges created using [claude.ia](https://claude.ai)

# Setup

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os
import sys

In [2]:
# Required for Spark to find Python executable
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17'

In [3]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("InterviewChallenges") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/19 16:28:34 WARN Utils: Your hostname, Alissons-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.19 instead (on interface en0)
25/07/19 16:28:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/19 16:28:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Challenge 1: Basic DataFrame Operations (Beginner)
Problem: Given a sales dataset, perform basic transformations and aggregations.

Expected Skills: DataFrame creation, column transformations, aggregations, filtering, sorting

In [11]:
# Sample data
sales_data = [
    ("John", "Electronics", 1200, "2023-01-15"),
    ("Alice", "Clothing", 800, "2023-01-16"),
    ("Bob", "Electronics", 1500, "2023-01-17"),
    ("Alice", "Books", 300, "2023-01-18"),
    ("John", "Clothing", 600, "2023-01-19"),
    ("Bob", "Books", 250, "2023-01-20")
]

schema = StructType([
    StructField("salesperson", StringType(), True),
    StructField("category", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("sale_date", StringType(), True)
])

sales_df = spark.createDataFrame(sales_data, schema)

## Convert sale_date from string to date type

In [16]:
sales_df = sales_df.withColumn('sale_date', F.col('sale_date').cast('date'))

## Find total sales amount by each salesperson

In [20]:
sales_amount_by_person_df = sales_df.groupby('salesperson').agg(
    F.sum(F.col('amount')).alias('total_amount')
)

In [21]:
sales_amount_by_person_df.show()

+-----------+------------+
|salesperson|total_amount|
+-----------+------------+
|       John|        1800|
|      Alice|        1100|
|        Bob|        1750|
+-----------+------------+



## Find the average sale amount by category

In [25]:
avg_amount_by_category_df = sales_df.groupby('category').agg(
    F.avg(F.col('amount')).alias('avg_amount')
)

In [26]:
avg_amount_by_category_df.show()

+-----------+----------+
|   category|avg_amount|
+-----------+----------+
|Electronics|    1350.0|
|   Clothing|     700.0|
|      Books|     275.0|
+-----------+----------+



## Show only sales where amount > 500, sorted by amount descending

In [28]:
(sales_df
 .where('amount > 500')
 .sort('amount', ascending=False).show())

+-----------+-----------+------+----------+
|salesperson|   category|amount| sale_date|
+-----------+-----------+------+----------+
|        Bob|Electronics|  1500|2023-01-17|
|       John|Electronics|  1200|2023-01-15|
|      Alice|   Clothing|   800|2023-01-16|
|       John|   Clothing|   600|2023-01-19|
+-----------+-----------+------+----------+



# Challenge 2: Window Functions (Intermediate)
Problem: Analyze employee salary data using window functions.

Expected Skills: Window functions, ranking, cumulative operations, lead/lag functions

In [29]:
# Employee salary data
emp_data = [
    (1, "Alice", "Engineering", 75000, "2020-01-01"),
    (2, "Bob", "Engineering", 80000, "2019-06-15"),
    (3, "Charlie", "Marketing", 65000, "2021-03-01"),
    (4, "Diana", "Engineering", 85000, "2018-09-01"),
    (5, "Eve", "Marketing", 70000, "2020-11-01"),
    (6, "Frank", "Sales", 60000, "2022-01-01"),
    (7, "Grace", "Sales", 62000, "2021-08-01")
]

emp_schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("hire_date", StringType(), True)
])

emp_df = spark.createDataFrame(emp_data, emp_schema)

## Rank employees by salary within each department

In [39]:
emp_rank_by_department_df = (emp_df
                             .withColumn('rank',
                                         F.rank().over(
                                             Window.partitionBy('department').orderBy('salary'))
                                         ))

In [40]:
emp_rank_by_department_df.show()

+------+-------+-----------+------+----------+----+
|emp_id|   name| department|salary| hire_date|rank|
+------+-------+-----------+------+----------+----+
|     1|  Alice|Engineering| 75000|2020-01-01|   1|
|     2|    Bob|Engineering| 80000|2019-06-15|   2|
|     4|  Diana|Engineering| 85000|2018-09-01|   3|
|     3|Charlie|  Marketing| 65000|2021-03-01|   1|
|     5|    Eve|  Marketing| 70000|2020-11-01|   2|
|     6|  Frank|      Sales| 60000|2022-01-01|   1|
|     7|  Grace|      Sales| 62000|2021-08-01|   2|
+------+-------+-----------+------+----------+----+



## Calculate running total of salaries ordered by hire date

In [35]:
running_total_salaries_df = emp_df.withColumn('hire_date', F.col('hire_date').cast('date'))

In [ ]:
running_total_salaries_df.withColumn('running_total_salary', F.sum('salary').over(Window.orderBy('hire_date'))).show()

## Find the salary difference between each employee and the highest-paid employee in their department

In [48]:
salary_difference_in_department_df = (emp_rank_by_department_df
                                      .withColumn('rank',
                                                  F.rank().over(
                                                      Window.partitionBy('department').orderBy(F.desc('salary')))
                                                  ))

In [60]:
salary_diff_window = Window.partitionBy('department').orderBy(F.desc('salary'))
salary_difference_in_department_df = salary_difference_in_department_df.withColumns({
    'is_highest_of_department': F.col('rank') == 1,
    'diff_preceding_salary': F.coalesce(
        F.col('salary') - F.lag('salary').over(salary_diff_window),
        F.lit(0)
    )
})

In [ ]:
salary_difference_in_department_df.show()

## Identify the 2nd highest salary in each department

In [62]:
second_highest_in_department_df = salary_difference_in_department_df.withColumn('is_second_highest_of_dept',
                                                                                F.col('rank') == 2)

In [ ]:
second_highest_in_department_df.where(F.col('is_second_highest_of_dept')).show()

# Challenge 3: Complex Joins and Data Quality (Intermediate-Advanced)

Problem: Join customer, order, and product data while handling data quality issues.

Expected Skills: Multiple join types, data quality checks, null handling, data validation

In [65]:
# Customer data
customers_data = [
    (1, "John Doe", "john@email.com", "New York"),
    (2, "Jane Smith", "jane@email.com", "California"),
    (3, "Bob Johnson", None, "Texas"),  # Missing email
    (4, "Alice Brown", "alice@email.com", "Florida")
]

# Orders data (some customers might not exist)
orders_data = [
    (101, 1, "2023-01-15", 250.0),
    (102, 2, "2023-01-16", 180.0),
    (103, 1, "2023-01-17", 320.0),
    (104, 5, "2023-01-18", 150.0),  # Customer ID 5 doesn't exist
    (105, 2, "2023-01-19", 200.0),
    (106, 3, "2023-01-20", 175.0)
]

# Product orders (order line items)
order_items_data = [
    (101, "P001", 2, 125.0),
    (102, "P002", 1, 180.0),
    (103, "P001", 1, 125.0),
    (103, "P003", 1, 195.0),
    (105, "P002", 2, 100.0),
    (106, "P001", 1, 125.0),
    (106, "P004", 1, 50.0)
]

customers_df = spark.createDataFrame(customers_data,
                                     ["customer_id", "name", "email", "state"])
orders_df = spark.createDataFrame(orders_data,
                                  ["order_id", "customer_id", "order_date", "total_amount"])
order_items_df = spark.createDataFrame(order_items_data,
                                       ["order_id", "product_id", "quantity", "price"])

## Identify and handle orphaned orders (orders without valid customers)

In [99]:
customers_orders_df = customers_df.join(orders_df, how='full', on='customer_id')
# joined_df = joined_df.join(order_items_df, how='full', on='order_id')

In [ ]:
customers_orders_df.show()

In [129]:
order_without_customers_df = customers_orders_df.where(customers_df.customer_id.isNull()).withColumns({
    'name': F.coalesce(F.col('name'), F.lit('Not registered')),
    'email': F.coalesce(F.col('email'), F.lit('Not registered')),
    'state': F.coalesce(F.col('state'), F.lit('Not registered'))
})

In [130]:
order_without_customers_df.show()

+-----------+--------------+--------------+--------------+--------+----------+------------+
|customer_id|          name|         email|         state|order_id|order_date|total_amount|
+-----------+--------------+--------------+--------------+--------+----------+------------+
|          5|Not registered|Not registered|Not registered|     104|2023-01-18|       150.0|
+-----------+--------------+--------------+--------------+--------+----------+------------+



## Find customers with no orders

In [121]:
(customers_orders_df
 .where(F.col('order_id').isNull())
 .select(['customer_id', 'name', 'email', 'state'])
 .show())

+-----------+-----------+---------------+-------+
|customer_id|       name|          email|  state|
+-----------+-----------+---------------+-------+
|          4|Alice Brown|alice@email.com|Florida|
+-----------+-----------+---------------+-------+



## Calculate total revenue by state, handling missing data appropriately

In [135]:
total_revenue_by_state_df = customers_orders_df.groupBy('state').agg(
    F.coalesce(F.sum('total_amount').alias('total_amount'), F.lit(0))
).withColumn('state', F.coalesce(F.col('state'), F.lit('Not registered')))

In [136]:
total_revenue_by_state_df.show()

+--------------+----------------------------------------------+
|         state|coalesce(sum(total_amount) AS total_amount, 0)|
+--------------+----------------------------------------------+
|         Texas|                                         175.0|
|Not registered|                                         150.0|
|       Florida|                                           0.0|
|    California|                                         380.0|
|      New York|                                         570.0|
+--------------+----------------------------------------------+



## Validate data consistency between orders and order_items tables

In [137]:
orders_items_df = orders_df.join(order_items_df, how="outer", on='order_id')

In [138]:
orders_items_df.show()

+--------+-----------+----------+------------+----------+--------+-----+
|order_id|customer_id|order_date|total_amount|product_id|quantity|price|
+--------+-----------+----------+------------+----------+--------+-----+
|     101|          1|2023-01-15|       250.0|      P001|       2|125.0|
|     102|          2|2023-01-16|       180.0|      P002|       1|180.0|
|     103|          1|2023-01-17|       320.0|      P001|       1|125.0|
|     103|          1|2023-01-17|       320.0|      P003|       1|195.0|
|     104|          5|2023-01-18|       150.0|      NULL|    NULL| NULL|
|     105|          2|2023-01-19|       200.0|      P002|       2|100.0|
|     106|          3|2023-01-20|       175.0|      P001|       1|125.0|
|     106|          3|2023-01-20|       175.0|      P004|       1| 50.0|
+--------+-----------+----------+------------+----------+--------+-----+



In [143]:
orders_items_grouped_df = orders_items_df.groupby('order_id', 'total_amount').agg(
    F.sum('quantity').alias('total_items'),
    F.sum(F.col('price') * F.col('quantity')).alias('total_amount_items')
).orderBy('order_id')

In [ ]:
orders_items_grouped_df.show()

In [147]:
orders_items_check_df = orders_items_grouped_df.withColumns({
    'values_match': F.coalesce(F.col('total_amount') == F.col('total_amount_items'), F.lit(False)),
    'has_items': F.col('total_items').isNotNull()
})

In [ ]:
orders_items_check_df.show()

# Challenge 4: Advanced Aggregations and Pivoting (Advanced)

Problem: Analyze time-series sales data with complex aggregations.

Expected Skills: Pivot operations, time-series analysis, complex window functions, OLAP operations

In [4]:
# Monthly sales data
monthly_sales_data = [
    ("2023-01", "Electronics", "North", 15000),
    ("2023-01", "Electronics", "South", 12000),
    ("2023-01", "Clothing", "North", 8000),
    ("2023-01", "Clothing", "South", 6000),
    ("2023-02", "Electronics", "North", 18000),
    ("2023-02", "Electronics", "South", 14000),
    ("2023-02", "Clothing", "North", 9000),
    ("2023-02", "Clothing", "South", 7500),
    ("2023-03", "Electronics", "North", 16000),
    ("2023-03", "Electronics", "South", 13500),
    ("2023-03", "Clothing", "North", 8500),
    ("2023-03", "Clothing", "South", 7000)
]

monthly_sales_df = spark.createDataFrame(monthly_sales_data,
                                         ["month", "category", "region", "sales"])

## Create a pivot table showing sales by month (rows) and region (columns)

In [ ]:
# This way it also works and gives flexibiity to apply more than one aggregation
# pivot_month_and_region_df = monthly_sales_df.groupBy('month').pivot('region').agg(F.sum("sales").alias("total_sales"))

pivot_month_and_region_df = monthly_sales_df.groupBy('month').pivot('region').sum("sales")

## Calculate month-over-month growth rate for each category-region combination

In [59]:
# Calculate month-over-month growth rate for each category-region combination

month_category_region_window = Window.partitionBy(
    ['category', 'region']).orderBy('month')

monthly_sales_df = monthly_sales_df.groupBy(['category', 'region', 'month']).agg(
    F.sum('sales').alias('sales')
) \
    .withColumns({
        'mom_growth': F.col('sales') - F.lag(F.col('sales'), 1).over(month_category_region_window),
        'mom_growth_rate': F.round((F.col('sales') / F.lag(F.col('sales'), 1).over(month_category_region_window) - 1) * 100, 2),
    }) \
    .orderBy(['category', 'region', 'month'])

monthly_sales_df.show()

+-----------+------+-------+-----+----------+---------------+
|   category|region|  month|sales|mom_growth|mom_growth_rate|
+-----------+------+-------+-----+----------+---------------+
|   Clothing| North|2023-01| 8000|      NULL|           NULL|
|   Clothing| North|2023-02| 9000|      1000|           12.5|
|   Clothing| North|2023-03| 8500|      -500|          -5.56|
|   Clothing| South|2023-01| 6000|      NULL|           NULL|
|   Clothing| South|2023-02| 7500|      1500|           25.0|
|   Clothing| South|2023-03| 7000|      -500|          -6.67|
|Electronics| North|2023-01|15000|      NULL|           NULL|
|Electronics| North|2023-02|18000|      3000|           20.0|
|Electronics| North|2023-03|16000|     -2000|         -11.11|
|Electronics| South|2023-01|12000|      NULL|           NULL|
|Electronics| South|2023-02|14000|      2000|          16.67|
|Electronics| South|2023-03|13500|      -500|          -3.57|
+-----------+------+-------+-----+----------+---------------+



In [62]:
# To make the result more concise it is possible to pivot
# region into columns:

monthly_sales_df.groupBy(['category', 'month']) \
    .pivot('region') \
    .agg(
        F.first('sales'),
        F.first('mom_growth').alias('mom_growth'),
        F.first('mom_growth_rate').alias('mom_growth_rate')
    ) \
    .orderBy(['category', 'month']) \
    .show()

+-----------+-------+------------------+----------------+---------------------+------------------+----------------+---------------------+
|   category|  month|North_first(sales)|North_mom_growth|North_mom_growth_rate|South_first(sales)|South_mom_growth|South_mom_growth_rate|
+-----------+-------+------------------+----------------+---------------------+------------------+----------------+---------------------+
|   Clothing|2023-01|              8000|            NULL|                 NULL|              6000|            NULL|                 NULL|
|   Clothing|2023-02|              9000|            1000|                 12.5|              7500|            1500|                 25.0|
|   Clothing|2023-03|              8500|            -500|                -5.56|              7000|            -500|                -6.67|
|Electronics|2023-01|             15000|            NULL|                 NULL|             12000|            NULL|                 NULL|
|Electronics|2023-02|             